In [1]:
# Import essential libraries for data analysis and mapping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins

# Census Geography and Spatial Analysis

In this section, we'll download census geography data for Philadelphia and join it with our bike share data. We'll use Python libraries equivalent to R's tidycensus package:

- `geopandas` for spatial data handling
- `cenpy` for downloading census data
- `contextily` for basemaps

In [ ]:
# Install and import additional libraries for census data and spatial analysis
import geopandas as gpd
import cenpy as cp
import contextily as ctx
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

# Set up cenpy (Census API client)
# You may need to get a free API key from https://api.census.gov/data/key_signup.html
# For now, we'll use the default without a key (rate limited)

print("Census and spatial libraries imported successfully!")
print("Available Census products:")
# Check available products (API may vary)
try:
    print(list(cp.products.__dict__.keys()))
except:
    print("ACS and other standard products available")

In [ ]:
# Download census tract geometries for Philadelphia County
# Philadelphia County FIPS code is 42101 (State: 42 = PA, County: 101 = Philadelphia)

# Method 1: Using cenpy (limited to 2017-2019)
try:
    philadelphia_tracts = cp.products.ACS(2019).from_county(
        'Philadelphia County, PA',
        level='tract',
        variables=['B01003_001E'],  # Total population variable
        return_geometry=True
    )
    philadelphia_tracts_simple = philadelphia_tracts[['GEOID', 'geometry']].copy()
    print("Successfully downloaded census tracts using cenpy")
    
except Exception as e:
    print(f"Cenpy method failed: {e}")
    print("Trying alternative method...")
    
    # Method 2: Direct download from Census Bureau
    # URL for 2020 census tracts (most recent available)
    url = "https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_42_tract.zip"
    
    try:
        # Read directly from URL
        philadelphia_tracts_all = gpd.read_file(url)
        # Filter for Philadelphia County (COUNTYFP = '101')
        philadelphia_tracts_simple = philadelphia_tracts_all[
            philadelphia_tracts_all['COUNTYFP'] == '101'
        ][['GEOID', 'geometry']].copy()
        print("Successfully downloaded census tracts from Census Bureau")
        
    except Exception as e2:
        print(f"Direct download also failed: {e2}")
        print("Creating sample tract geometries for demonstration...")
        # Create a simple bounding box for Philadelphia as fallback
        from shapely.geometry import Polygon
        philly_bounds = Polygon([
            (-75.28, 39.87), (-74.96, 39.87), 
            (-74.96, 40.14), (-75.28, 40.14)
        ])
        philadelphia_tracts_simple = gpd.GeoDataFrame({
            'GEOID': ['42101000100'], 
            'geometry': [philly_bounds]
        }, crs='EPSG:4326')
        print("Created sample geometry for demonstration")

print(f"Final result: {len(philadelphia_tracts_simple)} census tracts for Philadelphia")
print(f"Columns: {philadelphia_tracts_simple.columns.tolist()}")
print(f"CRS: {philadelphia_tracts_simple.crs}")

# Display first few rows
philadelphia_tracts_simple.head()

In [ ]:
# Load the Indego bike share data
try:
    # Try to read a sample of the data first to understand structure
    bike_data_sample = pd.read_csv('indego-trips-2025-q3.csv', nrows=1000)
    print(f"Columns: {bike_data_sample.columns.tolist()}")
    print(f"Sample shape: {bike_data_sample.shape}")
    print("\nFirst few rows:")
    print(bike_data_sample.head())
    
except Exception as e:
    print(f"Error reading data: {e}")
    print("The file might be too large. Let's try reading it in chunks.")

In [ ]:
# Function to perform spatial join between points and census tracts
def add_census_tract_info(df, lat_col, lon_col, tract_suffix=''):
    """
    Add census tract information to a dataframe with lat/lon coordinates
    
    Parameters:
    df: DataFrame with latitude and longitude columns
    lat_col: Name of latitude column
    lon_col: Name of longitude column
    tract_suffix: Suffix to add to GEOID column name
    
    Returns:
    DataFrame with added GEOID column
    """
    # Create geometry points from lat/lon
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    
    # Ensure both GeoDataFrames use the same CRS
    gdf = gdf.to_crs(philadelphia_tracts_simple.crs)
    
    # Clean any existing index columns that might conflict
    if 'index_right' in gdf.columns:
        gdf = gdf.drop(columns=['index_right'])
    
    # Perform spatial join
    joined = gpd.sjoin(gdf, philadelphia_tracts_simple, how='left', predicate='within')
    
    # Rename GEOID column if suffix provided
    if tract_suffix:
        joined = joined.rename(columns={'GEOID': f'GEOID_{tract_suffix}'})
    
    # Drop geometry and index columns to return regular DataFrame
    columns_to_drop = ['geometry']
    if 'index_right' in joined.columns:
        columns_to_drop.append('index_right')
    
    result = pd.DataFrame(joined.drop(columns=columns_to_drop))
    
    return result

print("Spatial join function defined successfully!")

In [ ]:
# Define the function to process bike data with tract information
def process_bike_data_with_tracts(df, start_lat_col, start_lon_col, end_lat_col, end_lon_col):
    """
    Process bike share data by adding origin and destination census tract information
    
    Parameters:
    df: DataFrame with bike share trip data
    start_lat_col: Name of start latitude column
    start_lon_col: Name of start longitude column  
    end_lat_col: Name of end latitude column
    end_lon_col: Name of end longitude column
    
    Returns:
    DataFrame with added GEOID_origin and GEOID_destination columns
    """
    print("Adding origin tract information...")
    # Add origin tract info
    df_with_origin = add_census_tract_info(df, start_lat_col, start_lon_col, 'origin')
    
    print("Adding destination tract information...")
    # Add destination tract info
    df_with_both = add_census_tract_info(df_with_origin, end_lat_col, end_lon_col, 'destination')
    
    return df_with_both

print("process_bike_data_with_tracts function defined successfully!")

In [ ]:
# Process the bike share data and add spatial information
# Assuming the bike share data has columns for origin and destination coordinates

if 'bike_data_sample' in locals():
    # Check what coordinate columns are available
    coord_columns = [col for col in bike_data_sample.columns if any(term in col.lower() for term in ['lat', 'lon', 'longitude', 'latitude', 'start', 'end'])]
    print("Potential coordinate columns found:")
    print(coord_columns)
    
    # Common column patterns for bike share data
    possible_patterns = {
        'start_lat': ['start_lat', 'start_latitude', 'from_lat'],
        'start_lon': ['start_lon', 'start_longitude', 'from_lon', 'start_lng'],
        'end_lat': ['end_lat', 'end_latitude', 'to_lat'],
        'end_lon': ['end_lon', 'end_longitude', 'to_lon', 'end_lng']
    }
    
    # Find actual column names
    actual_columns = {}
    for key, patterns in possible_patterns.items():
        for pattern in patterns:
            if pattern in bike_data_sample.columns:
                actual_columns[key] = pattern
                break
    
    print("\nMapped coordinate columns:")
    print(actual_columns)
    
    if len(actual_columns) >= 4:
        print("\nFound all required coordinate columns! Ready for spatial joining.")
    else:
        print("\nNeed to identify coordinate columns. Let's examine the data structure more closely.")
        print(bike_data_sample.info())

In [ ]:
# Now let's add spatial information to our bike share data
print("Processing bike share data with spatial joins...")

# Apply spatial joins to add origin and destination tract information
bike_data_with_tracts = process_bike_data_with_tracts(
    bike_data_sample, 
    'start_lat', 'start_lon', 
    'end_lat', 'end_lon'
)

print(f"Original data shape: {bike_data_sample.shape}")
print(f"Data with tract info shape: {bike_data_with_tracts.shape}")
print(f"New columns added: {set(bike_data_with_tracts.columns) - set(bike_data_sample.columns)}")

# Show sample of the enhanced data
print("\nSample of data with tract information:")
print(bike_data_with_tracts[['trip_id', 'start_lat', 'start_lon', 'end_lat', 'end_lon', 
                            'GEOID_origin', 'GEOID_destination']].head())

In [ ]:
# Fixed visualization of census tracts with proper projection
def create_tract_map():
    """
    Create an improved map showing Philadelphia census tracts with proper projection
    """
    # Create base map
    fig, ax = plt.subplots(1, 1, figsize=(15, 12))
    
    # Plot census tracts - use the original CRS first
    philadelphia_tracts_simple.plot(ax=ax, color='lightblue', edgecolor='black', alpha=0.7, linewidth=0.5)
    
    # Set the extent based on the tract boundaries
    bounds = philadelphia_tracts_simple.total_bounds
    ax.set_xlim(bounds[0], bounds[2])
    ax.set_ylim(bounds[1], bounds[3])
    
    ax.set_title('Philadelphia Census Tracts (408 Tracts)', fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Add grid for better visualization
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Map bounds: {bounds}")
    print(f"Number of tracts plotted: {len(philadelphia_tracts_simple)}")

# Also create a simpler folium map
def create_folium_tract_map():
    """
    Create an interactive map using folium
    """
    # Get the center point of Philadelphia
    bounds = philadelphia_tracts_simple.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Create folium map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Add census tracts to the map
    folium.GeoJson(
        philadelphia_tracts_simple.to_json(),
        style_function=lambda feature: {
            'fillColor': 'lightblue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['GEOID'],
            aliases=['Census Tract:'],
            localize=True
        )
    ).add_to(m)
    
    return m

print("Improved mapping functions defined!")
print("Run create_improved_tract_map() for static map or create_folium_tract_map() for interactive map")

In [ ]:
# Create the census tract map
create_tract_map()

In [ ]:
# Extract unique Indego station locations from the bike share data
def extract_station_locations(df):
    """
    Extract unique station locations from bike share data
    """
    # Get start stations
    start_stations = df[['start_station', 'start_lat', 'start_lon']].drop_duplicates()
    start_stations.columns = ['station_id', 'latitude', 'longitude']
    start_stations['station_type'] = 'start'
    
    # Get end stations
    end_stations = df[['end_station', 'end_lat', 'end_lon']].drop_duplicates()
    end_stations.columns = ['station_id', 'latitude', 'longitude']
    end_stations['station_type'] = 'end'
    
    # Combine and get unique stations
    all_stations = pd.concat([start_stations, end_stations], ignore_index=True)
    unique_stations = all_stations.groupby('station_id').agg({
        'latitude': 'first',
        'longitude': 'first'
    }).reset_index()
    
    return unique_stations

# Extract station locations
station_locations = extract_station_locations(bike_data_sample)
print(f"Found {len(station_locations)} unique Indego stations")
print("\nSample station locations:")
print(station_locations.head())

# Check for any missing coordinates
missing_coords = station_locations[(station_locations['latitude'].isna()) | (station_locations['longitude'].isna())]
if len(missing_coords) > 0:
    print(f"\nWarning: {len(missing_coords)} stations have missing coordinates")
else:
    print("\n✓ All stations have valid coordinates")

In [ ]:
# Create a comprehensive map with census tracts and Indego stations
def create_comprehensive_map():
    """
    Create a map showing both census tracts and Indego bike share stations
    """
    # Filter out stations with missing coordinates
    valid_stations = station_locations.dropna(subset=['latitude', 'longitude'])
    
    # Create base map
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    
    # Plot census tracts
    philadelphia_tracts_simple.plot(
        ax=ax, 
        color='lightblue', 
        edgecolor='gray', 
        alpha=0.6, 
        linewidth=0.5,
        label='Census Tracts'
    )
    
    # Plot Indego stations
    ax.scatter(
        valid_stations['longitude'], 
        valid_stations['latitude'],
        c='pink', 
        s=30, 
        alpha=0.8, 
        marker='o',
        label=f'Indego Stations ({len(valid_stations)})',
        zorder=5
    )
    
    # Set the extent based on the tract boundaries
    bounds = philadelphia_tracts_simple.total_bounds
    ax.set_xlim(bounds[0], bounds[2])
    ax.set_ylim(bounds[1], bounds[3])
    
    ax.set_title('Philadelphia Census Tracts & Indego Bike Share Stations', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Add legend
    ax.legend(loc='upper right')
    
    # Add grid for better visualization
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Map shows {len(philadelphia_tracts_simple)} census tracts and {len(valid_stations)} Indego stations")

# Create an interactive Folium map with both tracts and stations
def create_interactive_stations_map():
    """
    Create an interactive map using folium with census tracts and stations
    """
    # Filter out stations with missing coordinates
    valid_stations = station_locations.dropna(subset=['latitude', 'longitude'])
    
    # Get the center point of Philadelphia
    bounds = philadelphia_tracts_simple.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Create folium map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Add census tracts
    folium.GeoJson(
        philadelphia_tracts_simple.to_json(),
        style_function=lambda feature: {
            'fillColor': 'lightblue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['GEOID'],
            aliases=['Census Tract:'],
            localize=True
        )
    ).add_to(m)
    
    # Add Indego stations
    for idx, station in valid_stations.iterrows():
        folium.CircleMarker(
            location=[station['latitude'], station['longitude']],
            radius=4,
            popup=f"Station ID: {station['station_id']}",
            color='pink',
            fill=True,
            fillColor='red',
            fillOpacity=0.8
        ).add_to(m)
    
    # Add a custom legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 80px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <b>Legend</b><br>
    <i class="fa fa-circle" style="color:lightblue"></i> Census Tracts<br>
    <i class="fa fa-circle" style="color:red"></i> Indego Stations
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m


In [ ]:
# Create static map with census tracts and stations
create_comprehensive_map()

In [ ]:
# Create a simplified interactive map to ensure it displays
print("Creating simplified interactive map...")

# Filter out stations with missing coordinates
valid_stations = station_locations.dropna(subset=['latitude', 'longitude'])

# Get the center point of Philadelphia
bounds = philadelphia_tracts_simple.total_bounds
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

print(f"Map center: {center_lat:.4f}, {center_lon:.4f}")
print(f"Valid stations: {len(valid_stations)}")

# Create a simple folium map
simple_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Add just a few stations first to test
for idx, station in valid_stations.head(10).iterrows():
    folium.CircleMarker(
        location=[station['latitude'], station['longitude']],
        radius=6,
        popup=f"Station {station['station_id']}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.8
    ).add_to(simple_map)


In [ ]:
# Create and save the comprehensive map with all stations and census tracts
comprehensive_interactive_map = create_interactive_stations_map()
comprehensive_interactive_map.save('philadelphia_comprehensive_map.html')

In [ ]:
# Summary Analysis: Origin-Destination Flow by Census Tract
print("=== SPATIAL ANALYSIS SUMMARY ===")
print(f"Total census tracts in Philadelphia: {len(philadelphia_tracts_simple)}")
print(f"Bike share trips analyzed: {len(bike_data_with_tracts)}")

# Analyze coverage
origin_tracts = bike_data_with_tracts['GEOID_origin'].dropna().nunique()
destination_tracts = bike_data_with_tracts['GEOID_destination'].dropna().nunique()

print(f"\nOrigin tract coverage: {origin_tracts} tracts")
print(f"Destination tract coverage: {destination_tracts} tracts")

# Most popular origin and destination tracts
print(f"\nTop 5 Origin Tracts:")
origin_counts = bike_data_with_tracts['GEOID_origin'].value_counts().head()
print(origin_counts)

print(f"\nTop 5 Destination Tracts:")
dest_counts = bike_data_with_tracts['GEOID_destination'].value_counts().head()
print(dest_counts)

# Analyze intra vs inter-tract trips
same_tract_trips = (bike_data_with_tracts['GEOID_origin'] == bike_data_with_tracts['GEOID_destination']).sum()
total_trips = len(bike_data_with_tracts)
print(f"\nTrip Patterns:")
print(f"Intra-tract trips (same origin and destination tract): {same_tract_trips} ({same_tract_trips/total_trips*100:.1f}%)")
print(f"Inter-tract trips: {total_trips - same_tract_trips} ({(total_trips - same_tract_trips)/total_trips*100:.1f}%)")


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pathlib import Path

# --- Load CSV ---
bike_csv = pd.read_csv('./data/indego-trips-2025-q3.csv')
df = bike_csv.copy()

lower_cols = {c.lower(): c for c in df.columns}

time_candidates = ["start_time", "started_at", "starttime", "start_time_local", "start_time_utc", "starttime_local"]
station_candidates = ["start_station_id", "start_station", "start_station_code",
                      "from_station_id", "from_station_name", "start_station_name"]

time_col = next((lower_cols[c] for c in time_candidates if c in lower_cols), None)
station_col = next((lower_cols[c] for c in station_candidates if c in lower_cols), None)

# Aggregate to hourly demand per station
dt = pd.to_datetime(df[time_col], errors="coerce")
df = df.loc[dt.notna()].copy()
df["dt_hour"] = dt.dt.floor("H")
df["station"] = df[station_col].astype(str)

agg = (
    df.groupby(["station", "dt_hour"])
      .size()
      .reset_index(name="y")
      .sort_values(["station", "dt_hour"])
      .reset_index(drop=True)
)

# --- Basic temporal features ---
agg["hour"] = agg["dt_hour"].dt.hour
agg["dow"] = agg["dt_hour"].dt.dayofweek
agg["is_weekend"] = (agg["dow"] >= 5).astype(int)
agg["y_lag1"] = agg.groupby("station")["y"].shift(1).fillna(0.0)

# --- Define X/y for baseline model ---
X = agg[["hour", "is_weekend", "y_lag1"]].values
y = agg["y"].values

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, agg.index.values, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)
agg["y_pred"] = model.predict(X)

# --- Save examples as images ---
def save_df_image(df, path, title=""):
    fig, ax = plt.subplots(figsize=(10, 2.2 + 0.3 * len(df)))
    ax.axis("off")
    tbl = ax.table(cellText=df.values, colLabels=df.columns, loc="center")
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)
    tbl.scale(1, 1.25)
    if title:
        ax.set_title(title, pad=10, fontsize=10)
    fig.tight_layout()
    fig.savefig(path, dpi=200, bbox_inches="tight")
    plt.close(fig)

out_dir = Path(".")

stations_sample = np.random.choice(agg["station"].unique(), size=min(5, agg["station"].nunique()), replace=False)
sample_df = agg[agg["station"].isin(stations_sample)].groupby("station").head(2)

input_example = sample_df.loc[:, ["station", "dt_hour", "hour", "is_weekend", "y_lag1", "y"]].copy()
output_example = sample_df.loc[:, ["station", "dt_hour", "y", "y_pred"]].copy()

save_df_image(input_example, out_dir / "input_example.png", title="Input Example (first rows)")
save_df_image(output_example, out_dir / "output_example.png", title="Output Example (first rows)")

print("Saved input_example.png and output_example.png successfully!")

/var/folders/yc/zpdsc5lj4qv865wcrkddqk9w0000gn/T/ipykernel_25732/3496378202.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  bike_csv = pd.read_csv('./data/indego-trips-2025-q3.csv')
/var/folders/yc/zpdsc5lj4qv865wcrkddqk9w0000gn/T/ipykernel_25732/3496378202.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["dt_hour"] = dt.dt.floor("H")


Saved input_example.png and output_example.png successfully!
